1) Sensor Fusion (No Camera) synchronous

In [66]:
import glob
import os
import sys
import carla
import random
import time
import numpy as np
import cv2

SHOW_PREVIEW = False
IM_WIDTH = 640
IM_HEIGHT = 480
SECONDS_PER_EPISODE = 10

actor_list = []
collision_hist = []

# Function to process IMU data
def process_imu(imu_data):
    acceleration = imu_data.accelerometer
    gyroscope = imu_data.gyroscope
    compass = imu_data.compass
    print("Acceleration:", acceleration)
    print("Gyroscope:", gyroscope)
    print("Compass:", compass)

# Function to process LiDAR data
def lidar_callback(data):
    lidar_data = np.frombuffer(data.raw_data, dtype=np.dtype('f4'))
    print("LiDAR Data:", lidar_data)
    with open('lidar_data.txt', 'w') as file:
        file.write(str(lidar_data))

# Function to handle collisions
def collision_data(event):
    collision_hist.append(event)
    print("Collision Detected!")

try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(20)
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    bp = blueprint_library.filter('model3')[0]
    spawn_point = random.choice(world.get_map().get_spawn_points())
    fixed_spawn_point = carla.Transform(
    location=carla.Location(x=100, y=100, z=0.1),
    rotation=carla.Rotation(pitch=0, yaw=-90, roll=0)  # Adjust the rotation as needed
)  
    vehicle = world.spawn_actor(bp, fixed_spawn_point)
    #vehicle = world.spawn_actor(bp, spawn_point)
    vehicle.apply_control(carla.VehicleControl(throttle=0.4, steer=0.01))
    actor_list.append(vehicle)

    # Adding IMU sensor
    imu = blueprint_library.find('sensor.other.imu')
    spawn_point = carla.Transform(carla.Location(x=2.9, z=0.7))
    imu_sensor = world.spawn_actor(imu, spawn_point, attach_to=vehicle)
    actor_list.append(imu_sensor)
    imu_sensor.listen(lambda data: process_imu(data))

    # Adding LiDAR sensor
    lidar = blueprint_library.find('sensor.lidar.ray_cast')
    spawn_point = carla.Transform(carla.Location(x=5.6, z=0.7))
    lidar_sensor = world.spawn_actor(lidar, spawn_point, attach_to=vehicle)
    actor_list.append(lidar_sensor)
    lidar_sensor.listen(lidar_callback)

    # Adding Collision sensor
    col_sensor = blueprint_library.find('sensor.other.collision')
    spawn_point = carla.Transform(carla.Location(x=2.9, z=0.7))
    collision_sensor = world.spawn_actor(col_sensor, spawn_point, attach_to=vehicle)
    actor_list.append(collision_sensor)
    collision_sensor.listen(collision_data)

    # Enable synchronous mode with a fixed time step
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.1
    world.apply_settings(settings)

    # Run the simulation for a fixed duration (in synchronous mode)
    for _ in range(int(SECONDS_PER_EPISODE * 40)):
        world.tick()
        # Sensor fusion and decision-making logic can be implemented here
        # You can access data from collision_hist, IMU, and LiDAR in this section
        # Implement your desired logic for making decisions based on the sensor data

finally:
    print('Destroying actors')
    for actor in actor_list:
        actor.destroy()
    print('Done.')

    # Disable synchronous mode before exiting
    settings.synchronous_mode = False
    world.apply_settings(settings)

Acceleration: Vector3D(x=-0.000000, y=0.000000, z=9.810000)
Gyroscope: Vector3D(x=-0.000001, y=-0.000000, z=-0.000000)
Compass: 0.0
Acceleration: Vector3D(x=9709.997070, y=-9999.998047, z=-61.871181)
Gyroscope: Vector3D(x=-0.000001, y=-0.000000, z=-0.000000)
Compass: 0.0
Acceleration: Vector3D(x=0.000000, y=0.000000, z=10.137545)
Gyroscope: Vector3D(x=-0.000001, y=-0.000000, z=-0.000000)
Compass: 0.0
LiDAR Data: [ 4.5743165 -5.9521484  1.3236543 ...  0.1309375 -0.7032986  0.9943894]
LiDAR Data: [ 4.5737987  -5.951445    1.1498687  ...  0.08726563 -0.70234144
  0.99439704]
Acceleration: Vector3D(x=0.000000, y=0.000000, z=9.946362)
Gyroscope: Vector3D(x=0.000001, y=0.000000, z=0.000000)
Compass: 0.0
Acceleration: Vector3D(x=0.000000, y=0.000000, z=9.866426)
Gyroscope: Vector3D(x=-0.000001, y=0.000000, z=0.000000)
Compass: 0.0
LiDAR Data: [ 4.574326   -5.952129    1.3236531  ...  0.17399414 -0.7019485
  0.99440014]
Acceleration: Vector3D(x=0.000000, y=0.000000, z=9.838673)
Gyroscope: Vect

2. DDPG Implementation

In [ ]:
import glob
import os
import sys
import carla
import random
import time
import numpy as np
import cv2
import math

SHOW_CAM = False
IM_WIDTH = 640
IM_HEIGHT = 480
SECONDS_PER_EPISODE = 10

class CarEnv:
    # Define constants that are required
    STEER_AMT = 1.0
    im_width = IM_WIDTH
    im_height = IM_HEIGHT
    front_camera = None

    def __init__(self):
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(10)
        self.world = self.client.get_world()
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]

        self.collision_hist = []
        self.actor_list = []

        self.front_camera = None
        self.episode_start = 0

    def reset(self):
        self.collision_hist = []
        self.actor_list = []

        self.transform = random.choice(self.world.get_map().get_spawn_points())
        self.vehicle = self.world.spawn_actor(self.model_3, self.transform)
        self.actor_list.append(self.vehicle)

        self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
        self.rgb_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.rgb_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.rgb_cam.set_attribute("fov", f"110")
        transform = carla.Transform(carla.Location(x=2.5, z=0.7))
        self.sensor = self.world.spawn_actor(self.rgb_cam, transform, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)

        self.lidar_sensor = self.blueprint_library.find('sensor.lidar.ray_cast')
        transform = carla.Transform(carla.Location(x=2.0, z=0.9))
        self.sensor = self.world.spawn_actor(self.lidar_sensor, transform, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)

        self.imu_sensor = self.blueprint_library.find('sensor.other.imu')
        transform = carla.Transform(carla.Location(x=2.0, y=1.0, z=0.8))
        self.sensor = self.world.spawn_actor(self.imu_sensor, transform, attach_to=self.vehicle)

        self.gps_sensor = self.blueprint_library.find('sensor.other.gnss')
        transform = carla.Transform(carla.Location(x=1.5, z=0.7))
        self.sensor = self.world.spawn_actor(self.gps_sensor, transform, attach_to=self.vehicle)

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(4)

        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, transform, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))

        while self.front_camera is None:
            time.sleep(0.01)

        self.episode_start = time.time()
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))

        return self.front_camera

    def collision_data(self, event):
        self.collision_hist.append(event)

    def process_img(self, image):
        i = np.array(image.raw_data)
        i2 = i.reshape((self.im_height, self.im_width, 4))
        i3 = i2[:, :, :3]
        if SHOW_CAM:
            cv2.imshow("", i3)
            cv2.waitKey(1)
        self.front_camera = i3

    def step(self, action):
        if action == 0:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=-self.STEER_AMT))
        elif action == 1:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0))
        elif action == 2:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=self.STEER_AMT))

        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        if len(self.collision_hist) != 0:
            done = True
            reward = -200
        elif kmh < 50:
            done = False
            reward = -1
        else:
            done = False
            reward = 1

        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True

        return self.front_camera, reward, done, None 

In [ ]:
# Define the Actor neural network

# import necessary libraries -------------------check which are already downlaoded using pip list command

# import pandas as pd      ------------------------Download using
# import numpy as np       -----------------------pip install numpy     
# import matplotlib.pyplot as plt                 pip install matplotlib
# import seaborn as sns ------------------------ !pip install seaborn
# import tensorflow as tf     ------------------ !pip install tensorflow
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam



class ActorNetwork(tf.keras.Model): # inherting tf.keras.Model so that we don't have to write it again and again
    def __init__(self, n_actions): # n_actioins are the number of possible actions
        super(ActorNetwork, self).__init__() # calling the costructor of parent class
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(64, activation='relu')
        self.output_layer = Dense(n_actions, activation='softmax') # OUTPUT  neurons are equal to number of actions that has to be taken like if 2 then 1 for throttle and 1 for steering

    # call function is used to call the actor and pass it the state representation to return the output
    def call(self, state): 
        x = self.dense1(state)
        x = self.dense2(x)
        return self.output_layer(x)

# Define the Critic neural network
class CriticNetwork(tf.keras.Model):
    def __init__(self):
        super(CriticNetwork, self).__init__()
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(64, activation='relu')
        self.output_layer = Dense(1)

    # call the critic to evaluate the performance of actor network
    def call(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        return self.output_layer(x)

# Define the DRL Actor-Critic agent
class DRLActorCritic:
    def __init__(self, state_dim, n_actions, lr_actor=0.001, lr_critic=0.001):
        self.state_dim = state_dim
        self.n_actions = n_actions  

        self.actor = ActorNetwork(n_actions)
        self.critic = CriticNetwork()

        self.actor_optimizer = Adam(learning_rate=lr_actor)
        self.critic_optimizer = Adam(learning_rate=lr_critic)

    def get_action(self, state): # it will take state generated by reset function, pass it to the actor network 
        #and returns an action Get action probabilities from the actor network
        
        action_probs = self.actor(np.array([state]))
        action = np.random.choice(self.n_actions, p=action_probs.numpy()[0])
        return action, action_probs

    def train(self, states, actions, discounted_rewards): 
        '''
          It will take state generated by the reset function, action predicted by the actor network and its cummulative reward
          
         it should take state and action taken by actor and use them to calculate max cummulative reward and use temporal
         difference to update the weights of actor and its own network
        
        it is for training of both the neural networks and does not return any value as such
        '''
        with tf.GradientTape() as actor_tape, tf.GradientTape() as critic_tape:
            # Convert inputs to TensorFlow tensors
            states = tf.convert_to_tensor(states, dtype=tf.float32)
            actions = tf.convert_to_tensor(actions, dtype=tf.int32)
            discounted_rewards = tf.convert_to_tensor(discounted_rewards, dtype=tf.float32)

            # Get action probabilities from the actor
            action_probs = self.actor(states)

            # Use actions to index into action_probs and calculate log probabilities
            selected_action_probs = tf.gather(action_probs, actions, batch_dims=1)
            log_action_probs = tf.math.log(selected_action_probs)

            # Calculate actor loss (policy gradient)
            actor_loss = -tf.reduce_sum(log_action_probs * discounted_rewards)

            # Calculate critic loss (mean squared error)
            state_values = self.critic(states)
            critic_loss = tf.reduce_mean(tf.square(discounted_rewards - state_values))

        # Compute gradients
        actor_gradients = actor_tape.gradient(actor_loss, self.actor.trainable_variables)
        critic_gradients = critic_tape.gradient(critic_loss, self.critic.trainable_variables)

        # Apply gradients to update actor and critic
        self.actor_optimizer.apply_gradients(zip(actor_gradients, self.actor.trainable_variables))
        self.critic_optimizer.apply_gradients(zip(critic_gradients, self.critic.trainable_variables)

        # Return the actor and critic loss for monitoring
        return actor_loss, critic_loss

In [ ]:
if __name__ == '__main__':
    vehicle = CarEnv()
    initial_state = vehicle.reset()
    n_actions = 2
    state_dim = initial_state.shape[0]  # Use shape[0] to get the number of features in the state
    agent = DRLActorCritic(state_dim, n_actions)

    max_steps = 100  # You should define the value of 'max_steps' somewhere

    # Perform multiple steps within the episode
    for t in range(max_steps):
        action, action_probs = agent.get_action(initial_state)  # The agent selects an action
        next_state, reward, done, info = vehicle.step(action)  # Perform a step in the environment
        

        if done:
            break  # The episode is done, and you can reset the environment for the next episode if needed
        else:
            initial_state = next_state  # Update the state for the next step

1. Check step function
2. make proper reward function
3. deal with state representation of the environment (camera data)
4. deal with synchronouse mode in reset function